# Day12_2: 모델 평가 (Model Evaluation) - 정답

---

In [ ]:
# 필수 라이브러리 임포트
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from sklearn.model_selection import train_test_split, cross_val_score, learning_curve
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    confusion_matrix, classification_report,
    mean_absolute_error, mean_squared_error, r2_score
)
from sklearn.datasets import load_breast_cancer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler

import warnings
warnings.filterwarnings('ignore')

# 데이터 로드
cancer = load_breast_cancer()
X = pd.DataFrame(cancer.data, columns=cancer.feature_names)
y = cancer.target

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

print("라이브러리 로드 완료!")

---

## Q1. 정확도 계산하기 ⭐

**문제**: 다음 실제값과 예측값으로 정확도(Accuracy)를 계산하세요.

In [ ]:
y_true = [1, 0, 1, 1, 0, 1, 0, 0, 1, 1]
y_pred = [1, 0, 1, 0, 0, 1, 1, 0, 1, 0]

# 정답 코드
accuracy = accuracy_score(y_true, y_pred)
print(f"정확도: {accuracy:.2f} ({accuracy*100:.0f}%)")

In [ ]:
# 테스트
assert accuracy == 0.7, "정확도는 0.7이어야 합니다"
print("테스트 통과!")

### 💡 풀이 설명

**접근 방법**:
- sklearn의 accuracy_score() 함수 사용
- 실제값과 예측값 리스트를 인자로 전달

**핵심 개념**:
- 정확도 = 맞은 예측 / 전체 예측
- 10개 중 7개 맞음 = 0.7

**대안**:
```python
# 수동 계산
correct = sum([t == p for t, p in zip(y_true, y_pred)])
accuracy = correct / len(y_true)
```

**흔한 실수**:
- y_true와 y_pred 순서 바꾸기 (결과는 같지만 일관성 중요)

**실무 팁**:
- 정확도만으로 모델 평가하지 말 것
- 불균형 데이터에서는 다른 지표 함께 확인

---

## Q2. 혼동행렬 해석하기 ⭐⭐

**문제**: Q1의 데이터로 혼동행렬을 만들고, TP, FP, FN, TN 값을 출력하세요.

In [ ]:
y_true = [1, 0, 1, 1, 0, 1, 0, 0, 1, 1]
y_pred = [1, 0, 1, 0, 0, 1, 1, 0, 1, 0]

# 정답 코드
cm = confusion_matrix(y_true, y_pred)
print("혼동행렬:")
print(cm)

tn, fp, fn, tp = cm.ravel()
print(f"\nTN (True Negative): {tn}")
print(f"FP (False Positive): {fp}")
print(f"FN (False Negative): {fn}")
print(f"TP (True Positive): {tp}")

In [ ]:
# 테스트
assert tn == 3, "TN은 3이어야 합니다"
assert fp == 1, "FP는 1이어야 합니다"
assert fn == 2, "FN은 2이어야 합니다"
assert tp == 4, "TP는 4이어야 합니다"
print("테스트 통과!")

### 💡 풀이 설명

**접근 방법**:
1. confusion_matrix()로 2x2 행렬 생성
2. ravel()로 1차원 배열로 변환
3. TN, FP, FN, TP 순서로 언패킹

**핵심 개념**:
- 혼동행렬 배치: [[TN, FP], [FN, TP]]
- ravel() 순서: TN, FP, FN, TP (왼쪽→오른쪽, 위→아래)

**대안**:
```python
tn = cm[0, 0]
fp = cm[0, 1]
fn = cm[1, 0]
tp = cm[1, 1]
```

**흔한 실수**:
- ravel() 순서 헷갈림 (TN, FP, FN, TP)
- TP와 TN 위치 혼동

**실무 팁**:
- 항상 ravel() 순서 확인: TN, FP, FN, TP

---

## Q3. Precision/Recall 계산하기 ⭐⭐

**문제**: Q1의 데이터로 Precision과 Recall을 계산하세요.

In [ ]:
y_true = [1, 0, 1, 1, 0, 1, 0, 0, 1, 1]
y_pred = [1, 0, 1, 0, 0, 1, 1, 0, 1, 0]

# 정답 코드
precision = precision_score(y_true, y_pred)
recall = recall_score(y_true, y_pred)

print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")

In [ ]:
# 테스트
assert precision == 0.8, "Precision은 0.8이어야 합니다"
assert recall == 2/3, "Recall은 약 0.67이어야 합니다"
print("테스트 통과!")

### 💡 풀이 설명

**접근 방법**:
- precision_score(), recall_score() 함수 사용

**핵심 개념**:
- Precision = TP / (TP + FP) = 4 / (4 + 1) = 0.8
- Recall = TP / (TP + FN) = 4 / (4 + 2) = 0.67

**대안**:
```python
# 수동 계산 (Q2에서 구한 값 활용)
precision = tp / (tp + fp)
recall = tp / (tp + fn)
```

**흔한 실수**:
- Precision과 Recall 공식 혼동
- 분모에 TN 포함 (잘못됨)

**실무 팁**:
- Precision: "양성 예측 중 진짜 양성"
- Recall: "실제 양성 중 찾아낸 비율"

---

## Q4. 회귀 지표 계산하기 ⭐⭐

**문제**: 다음 실제값과 예측값으로 MAE, RMSE, R-squared를 계산하세요.

In [ ]:
y_true_reg = [100, 150, 200, 180, 220]
y_pred_reg = [110, 140, 195, 175, 230]

# 정답 코드
mae = mean_absolute_error(y_true_reg, y_pred_reg)
mse = mean_squared_error(y_true_reg, y_pred_reg)
rmse = np.sqrt(mse)
r2 = r2_score(y_true_reg, y_pred_reg)

print(f"MAE: {mae:.2f}")
print(f"RMSE: {rmse:.2f}")
print(f"R-squared: {r2:.4f}")

In [ ]:
# 테스트
assert mae == 8.0, "MAE는 8.0이어야 합니다"
assert round(rmse, 2) == 8.37, "RMSE는 약 8.37이어야 합니다"
print("테스트 통과!")

### 💡 풀이 설명

**접근 방법**:
- sklearn.metrics의 mean_absolute_error, mean_squared_error, r2_score 사용
- RMSE는 MSE에 np.sqrt() 적용

**핵심 개념**:
- MAE = 평균(|실제 - 예측|) = (10+10+5+5+10)/5 = 8.0
- MSE = 평균((실제-예측)^2)
- RMSE = sqrt(MSE)
- R^2 = 모델의 설명력

**대안**:
```python
# 수동 계산
errors = np.array(y_true_reg) - np.array(y_pred_reg)
mae = np.mean(np.abs(errors))
rmse = np.sqrt(np.mean(errors**2))
```

**흔한 실수**:
- MSE와 RMSE 혼동
- np.sqrt 빼먹기

**실무 팁**:
- RMSE는 원래 단위로 해석 가능
- 이 예제: "평균 8만원 정도 오차"

---

## Q5. F1-score 수동 계산하기 ⭐⭐

**문제**: Precision=0.8, Recall=0.6일 때 F1-score를 수동으로 계산하세요.

In [ ]:
precision = 0.8
recall = 0.6

# 정답 코드
f1 = 2 * (precision * recall) / (precision + recall)
print(f"F1-score: {f1:.4f}")

In [ ]:
# 테스트
expected_f1 = 2 * 0.8 * 0.6 / (0.8 + 0.6)
assert abs(f1 - expected_f1) < 0.0001, "F1 계산이 맞지 않습니다"
print(f"테스트 통과! F1 = {f1:.4f}")

### 💡 풀이 설명

**접근 방법**:
- F1 = 2 * (P * R) / (P + R) 공식 적용

**핵심 개념**:
- F1-score는 Precision과 Recall의 조화 평균
- 조화 평균은 두 값 중 작은 쪽에 더 영향 받음
- 0.8과 0.6의 조화평균 = 0.6857 (산술평균 0.7보다 작음)

**대안**:
```python
# scipy 사용
from scipy.stats import hmean
f1 = hmean([precision, recall])
```

**흔한 실수**:
- 산술 평균 사용 (P + R) / 2 (잘못됨)
- 공식에서 2 빼먹기

**실무 팁**:
- F1이 조화평균이므로 P, R 중 하나만 높으면 F1은 낮음
- 균형 잡힌 모델이 높은 F1

---

## Q6. 교차검증 수행하기 ⭐⭐⭐

**문제**: 유방암 데이터에 대해 RandomForestClassifier로 5-fold 교차검증을 수행하고, 평균 정확도와 표준편차를 출력하세요.

In [ ]:
# 정답 코드
model = RandomForestClassifier(n_estimators=100, random_state=42)

cv_scores = cross_val_score(model, X_scaled, y, cv=5, scoring='accuracy')

print(f"각 Fold 점수: {cv_scores.round(4)}")
print(f"평균 정확도: {cv_scores.mean():.4f}")
print(f"표준편차: {cv_scores.std():.4f}")
print(f"95% 신뢰구간: {cv_scores.mean():.4f} (+/- {cv_scores.std()*2:.4f})")

In [ ]:
# 테스트
assert len(cv_scores) == 5, "5개 fold 결과가 있어야 합니다"
assert cv_scores.mean() > 0.9, "평균 정확도가 90% 이상이어야 합니다"
print("테스트 통과!")

### 💡 풀이 설명

**접근 방법**:
1. RandomForestClassifier 모델 생성
2. cross_val_score()에 모델, 데이터, cv=5, scoring 전달
3. mean(), std()로 통계 계산

**핵심 개념**:
- K-fold: 데이터를 K개로 나눠 K번 평가
- 각 fold에서 1/K는 테스트, 나머지는 학습
- 평균/표준편차로 안정성 확인

**대안**:
```python
# StratifiedKFold로 직접 구현
from sklearn.model_selection import StratifiedKFold
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
scores = []
for train_idx, val_idx in skf.split(X, y):
    model.fit(X_scaled[train_idx], y[train_idx])
    scores.append(model.score(X_scaled[val_idx], y[val_idx]))
```

**흔한 실수**:
- cv 파라미터 누락
- scoring 파라미터 잘못 지정

**실무 팁**:
- 표준편차가 크면 모델 불안정
- 보통 5-fold 또는 10-fold 사용

---

## Q7. 지표 선택 논리 ⭐⭐⭐

**문제**: 아래 상황에서 어떤 지표를 우선해야 하는지 답하고, 그 이유를 설명하세요.

In [ ]:
# 정답
print("1. 코로나 진단 키트: Recall 우선")
print("   이유: 실제 감염자를 놓치면(FN) 전파 위험")
print("         양성 오판(FP)은 재검사로 해결 가능")
print()
print("2. 뉴스 추천 시스템: Precision 우선")
print("   이유: 관심 없는 뉴스 추천(FP)은 사용자 경험 저하")
print("         좋은 뉴스를 못 추천(FN)은 상대적으로 덜 심각")
print()
print("3. 주가 예측 모델: RMSE 우선")
print("   이유: 큰 오차가 더 큰 손실로 이어짐")
print("         MAE는 큰 오차와 작은 오차를 동등 취급")

### 💡 풀이 설명

**접근 방법**:
- FP(오탐) vs FN(미탐) 비용 비교
- 비즈니스 맥락에서 어떤 오류가 더 치명적인지 판단

**핵심 개념**:
| 상황 | FP 비용 | FN 비용 | 우선 지표 |
|------|---------|---------|----------|
| 의료진단 | 재검사 비용 | 생명 위험 | Recall |
| 스팸필터 | 중요메일 손실 | 스팸 받음 | Precision |
| 추천시스템 | 사용자 불만 | 기회 손실 | Precision |

**실무 팁**:
- 항상 비즈니스 임팩트 먼저 고려
- 이해관계자와 FP/FN 비용 논의

---

## Q8. 혼동행렬 시각화하기 ⭐⭐⭐⭐

**문제**: 다음 혼동행렬을 Plotly 히트맵으로 시각화하세요.

In [ ]:
cm = np.array([[85, 15], [10, 90]])
labels = ['Negative', 'Positive']

# 정답 코드
# 텍스트 어노테이션 생성
annotations = []
for i in range(len(cm)):
    for j in range(len(cm[0])):
        annotations.append(
            dict(
                x=labels[j], y=labels[i],
                text=str(cm[i][j]),
                showarrow=False,
                font=dict(size=20, color='white' if cm[i][j] > cm.max()/2 else 'black')
            )
        )

fig = go.Figure(data=go.Heatmap(
    z=cm,
    x=labels,
    y=labels,
    colorscale='Blues',
    showscale=True
))

fig.update_layout(
    title='Confusion Matrix',
    xaxis_title='Predicted',
    yaxis_title='Actual',
    annotations=annotations,
    width=500, height=450
)

fig.show()

In [ ]:
# 테스트 (그래프 생성 확인)
assert fig is not None, "Figure가 생성되어야 합니다"
assert len(fig.data) > 0, "히트맵 데이터가 있어야 합니다"
print("테스트 통과!")

### 💡 풀이 설명

**접근 방법**:
1. go.Heatmap으로 기본 히트맵 생성
2. annotations로 셀 값 표시
3. 색상에 따라 텍스트 색상 조절 (가독성)

**핵심 개념**:
- z: 행렬 값
- x, y: 축 레이블
- colorscale: 색상 팔레트
- annotations: 셀 내 텍스트

**대안**:
```python
# Plotly Express로 간단히
fig = px.imshow(cm, x=labels, y=labels, 
                color_continuous_scale='Blues',
                text_auto=True)
```

**흔한 실수**:
- annotations 리스트 생성 안함
- x, y 인덱스 순서 혼동

**실무 팁**:
- 대각선 값이 크면 좋은 모델
- 어두운 배경에 흰 글씨로 가독성 향상

---

## Q9. Learning Curve 그리기 ⭐⭐⭐⭐

**문제**: 유방암 데이터에 대해 DecisionTreeClassifier의 Learning Curve를 그리고, 과적합 여부를 판단하세요.

In [ ]:
# 정답 코드
model = DecisionTreeClassifier(random_state=42)

train_sizes, train_scores, val_scores = learning_curve(
    model, X_scaled, y,
    train_sizes=np.linspace(0.1, 1.0, 10),
    cv=5,
    scoring='accuracy'
)

train_mean = train_scores.mean(axis=1)
train_std = train_scores.std(axis=1)
val_mean = val_scores.mean(axis=1)
val_std = val_scores.std(axis=1)

# 시각화
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=train_sizes, y=train_mean,
    mode='lines+markers',
    name='학습 점수',
    line=dict(color='blue')
))

fig.add_trace(go.Scatter(
    x=train_sizes, y=val_mean,
    mode='lines+markers',
    name='검증 점수',
    line=dict(color='green')
))

fig.update_layout(
    title='Learning Curve - Decision Tree',
    xaxis_title='학습 데이터 크기',
    yaxis_title='정확도',
    width=700, height=450
)

fig.show()

# 과적합 판단
gap = train_mean[-1] - val_mean[-1]
print(f"\n최종 학습 점수: {train_mean[-1]:.4f}")
print(f"최종 검증 점수: {val_mean[-1]:.4f}")
print(f"학습-검증 갭: {gap:.4f}")
print(f"\n판단: {'과적합 의심 (갭 > 0.05)' if gap > 0.05 else '양호'}")

In [ ]:
# 테스트
assert len(train_sizes) == 10, "10개 구간이어야 합니다"
assert train_mean[-1] > val_mean[-1], "Decision Tree는 보통 과적합 경향"
print("테스트 통과!")

### 💡 풀이 설명

**접근 방법**:
1. learning_curve()로 학습/검증 점수 계산
2. mean(axis=1)로 각 크기별 평균
3. Plotly로 두 곡선 시각화
4. 갭으로 과적합 판단

**핵심 개념**:
- 학습 점수 >> 검증 점수: 과적합
- 둘 다 낮음: 과소적합
- 갭이 좁고 둘 다 높음: 좋은 모델

**흔한 실수**:
- axis=1 빼먹어 잘못된 평균
- 학습/검증 곡선 혼동

**실무 팁**:
- Decision Tree는 기본적으로 과적합 경향
- max_depth 제한으로 해결

---

## Q10. 종합 모델 평가 대시보드 ⭐⭐⭐⭐⭐

**문제**: 유방암 데이터에 대해 3개 모델의 성능을 비교하는 대시보드를 만드세요.

In [ ]:
# 정답 코드

# 1. 모델 정의
models = {
    'Logistic Regression': LogisticRegression(random_state=42, max_iter=1000),
    'Random Forest': RandomForestClassifier(n_estimators=100, random_state=42),
    'SVM': SVC(random_state=42)
}

# 2. 교차검증 수행
results = []
for name, model in models.items():
    cv_acc = cross_val_score(model, X_scaled, y, cv=5, scoring='accuracy')
    cv_prec = cross_val_score(model, X_scaled, y, cv=5, scoring='precision')
    cv_rec = cross_val_score(model, X_scaled, y, cv=5, scoring='recall')
    cv_f1 = cross_val_score(model, X_scaled, y, cv=5, scoring='f1')
    
    results.append({
        'Model': name,
        'Accuracy': cv_acc.mean(),
        'Precision': cv_prec.mean(),
        'Recall': cv_rec.mean(),
        'F1-Score': cv_f1.mean()
    })

# 3. DataFrame 생성
results_df = pd.DataFrame(results)
print("모델 성능 비교표:")
print(results_df.round(4).to_string(index=False))

In [ ]:
# 4. 시각화
metrics = ['Accuracy', 'Precision', 'Recall', 'F1-Score']
colors = ['steelblue', 'darkorange', 'green', 'red']

fig = go.Figure()

for i, metric in enumerate(metrics):
    fig.add_trace(go.Bar(
        name=metric,
        x=results_df['Model'],
        y=results_df[metric],
        marker_color=colors[i]
    ))

fig.update_layout(
    title='모델 성능 비교 대시보드',
    xaxis_title='모델',
    yaxis_title='점수',
    barmode='group',
    yaxis=dict(range=[0.9, 1.0]),
    width=800, height=450,
    legend=dict(orientation='h', yanchor='bottom', y=1.02)
)

fig.show()

In [ ]:
# 5. 최고 모델 선정
best_model = results_df.loc[results_df['F1-Score'].idxmax(), 'Model']
best_f1 = results_df['F1-Score'].max()
print(f"\n최고 F1-Score 모델: {best_model} ({best_f1:.4f})")

In [ ]:
# 테스트
assert len(results_df) == 3, "3개 모델 결과가 있어야 합니다"
assert all(col in results_df.columns for col in ['Accuracy', 'Precision', 'Recall', 'F1-Score']), "모든 지표가 있어야 합니다"
assert results_df['Accuracy'].min() > 0.9, "모든 모델 정확도 90% 이상"
print("테스트 통과!")

### 💡 풀이 설명

**접근 방법**:
1. 모델 딕셔너리로 정의
2. 반복문으로 4개 지표 교차검증
3. 결과를 DataFrame으로 정리
4. grouped bar chart로 시각화
5. 최고 모델 선정

**핵심 개념**:
- 여러 지표를 종합적으로 비교
- 교차검증으로 안정성 확보
- 시각화로 직관적 비교

**대안**:
```python
# cross_validate로 한번에
from sklearn.model_selection import cross_validate
scoring = ['accuracy', 'precision', 'recall', 'f1']
cv_results = cross_validate(model, X, y, cv=5, scoring=scoring)
```

**흔한 실수**:
- scoring 이름 오타 ('f1_score' -> 'f1')
- 결과 정리 시 mean() 빼먹기

**실무 팁**:
- F1-Score로 최종 선택하되, 비즈니스 맥락 고려
- 지표 간 트레이드오프 분석
- 표준편차도 함께 보고해 안정성 확인

---

## 📊 학습 정리

### Part 1: 분류 평가 지표 핵심 요약

| 지표 | 공식 | 언제 사용? |
|------|------|----------|
| **Accuracy** | (TP+TN) / 전체 | 클래스 균형 시 |
| **Precision** | TP / (TP+FP) | FP 비용이 클 때 (스팸) |
| **Recall** | TP / (TP+FN) | FN 비용이 클 때 (암진단) |
| **F1-Score** | 조화평균(P,R) | 불균형 데이터 |
| **Confusion Matrix** | 2x2 표 | 오분류 패턴 분석 |

### Part 2: 회귀 평가 지표 및 과적합 방지 핵심 요약

| 지표/기법 | 특징 | 활용 |
|----------|------|------|
| **MAE** | 절대 오차 평균 | 이상치에 덜 민감 |
| **RMSE** | 제곱 오차 -> 루트 | 큰 오차 패널티 |
| **R-squared** | 설명력 (0~1) | 모델 비교 |
| **교차검증** | K-fold 평균 | 안정적 성능 추정 |
| **Learning Curve** | 학습량 vs 성능 | 과적합 진단 |

### 💡 실무 팁

1. **정확도 함정**: 불균형 데이터에서 정확도만 보면 안됨
2. **비용 기반 선택**: FP/FN 비용을 고려해 Precision/Recall 선택
3. **교차검증 필수**: 단일 분할 결과는 불안정
4. **Learning Curve**: 과적합 의심 시 반드시 확인
5. **여러 지표 종합**: 하나의 지표만 보지 말고 종합 판단